In [4]:
ls

Test_AI_2022_en.pdf*  test_set_data.csv*  train_set_data.csv*


In [5]:
import pandas as pd
import numpy as np

In [6]:
df_train = pd.read_csv("train_set_data.csv")

In [7]:
df_train.head()

,SMILES,Activity
0,Nc1ccccc1NC(=O)c2ccc(CNc3nccc(n3)c4cccnc4)cc2,2.28
1,CCNC(=O)c1cc2c(c(cnc2[nH]1)c3cncc(c3)C(=O)O)n4...,-0.99
2,Cc1cc(Cl)ccc1OC2CCN(C[C@H](O)CNC(=O)C3=CNC(=O)...,2.37
3,COc1ccc(cc1OC)C(C)NC(=O)C2(N)CCN(CC2)c3ncnc4[n...,1.80
4,C[C@@H](N)COc1c(C)cccc1C,0.60


In [8]:
df_train.shape

(3351, 2)

In [9]:
df_test = pd.read_csv("test_set_data.csv")

In [10]:
df_test.head()

,SMILES
0,C[C@H]1O[C@H]([C@H](O)[C@@H]1O)n2cnc3c(N)nc(Oc...
1,CCCCNc1cc(cc(c1Oc2ccccc2)S(=O)(=O)N)C(=O)O
2,FC(F)(F)c1ccc2[nH]c3CCN(Cc3c2c1)C(=O)[C@@H]4CC...
3,CCN(CC)CCNC(=O)c1c(C)[nH]c(\C=C\2/C(=O)Nc3ccc(...
4,OC1(CCN(CCCC(=O)c2ccc(F)cc2)CC1)c3ccc(Cl)cc3


In [11]:
df_test.shape

(837, 1)

we have to predict the activity of the molecules of the test set, it's a regression task.

In [12]:
from rdkit import Chem
from rdkit.Chem import AllChem

In [13]:
activity = df_train["Activity"]

In [14]:
activity

0       2.28
1      -0.99
2       2.37
3       1.80
4       0.60
        ... 
3346    4.20
3347    2.06
3348    1.90
3349    1.80
3350    1.54
Name: Activity, Length: 3351, dtype: float64

In [15]:
jit_activity = pd.DataFrame(activity)

In [16]:
jit_activity.shape

(3351, 1)

In [31]:
jit_activity.head()

,Activity
0,2.28
1,-0.99
2,2.37
3,1.80
4,0.60


In [17]:
train_smiles = df_train['SMILES']
test_smiles = df_test['SMILES']

In [18]:
train_smiles

0           Nc1ccccc1NC(=O)c2ccc(CNc3nccc(n3)c4cccnc4)cc2
1       CCNC(=O)c1cc2c(c(cnc2[nH]1)c3cncc(c3)C(=O)O)n4...
2       Cc1cc(Cl)ccc1OC2CCN(C[C@H](O)CNC(=O)C3=CNC(=O)...
3       COc1ccc(cc1OC)C(C)NC(=O)C2(N)CCN(CC2)c3ncnc4[n...
4                                C[C@@H](N)COc1c(C)cccc1C
                              ...                        
3346    COc1cc(ccc1Nc2ncc(Cl)c(n2)c3cnc4cc(Cl)ccn34)N5...
3347    CN1CCN(CC1)c2ccc3N=CN(C(=O)c3c2)c4cc(NC(=O)c5c...
3348                            COc1cnc(nc1N(C)C)c2ccccn2
3349    CN(C1CCN(Cc2ccc(nc2)C(F)(F)F)CC1)C(=O)Cc3ccc(c...
3350              CCSc1ccc(cc1)c2cc(ccc2OCC(=O)O)C(F)(F)F
Name: SMILES, Length: 3351, dtype: object

In [19]:
train_fps = [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smiles), 2) for smiles in train_smiles]
test_fps = [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smiles), 2) for smiles in test_smiles]

In [20]:
train_fps = np.asarray(train_fps)
test_fps = np.asarray(test_fps)

In [21]:
train_fps.shape

(3351, 2048)

In [22]:
test_fps.shape

(837, 2048)

In [23]:
jit_train = pd.DataFrame(train_fps)

In [24]:
jit_train.shape

(3351, 2048)

In [25]:
import pandas as pd

# Assuming your DataFrame is named "jit_train"
num_columns = 2048

# Create a list of column names
column_names = [f'fp{i+1}' for i in range(num_columns)]

# Assign the column names to the DataFrame
jit_train.columns = column_names

# Verify the column names
print(jit_train.columns)


Index(['fp1', 'fp2', 'fp3', 'fp4', 'fp5', 'fp6', 'fp7', 'fp8', 'fp9', 'fp10',
       ...
       'fp2039', 'fp2040', 'fp2041', 'fp2042', 'fp2043', 'fp2044', 'fp2045',
       'fp2046', 'fp2047', 'fp2048'],
      dtype='object', length=2048)


In [26]:
jit_train.head()

,fp1,fp2,fp3,fp4,fp5,fp6,fp7,fp8,fp9,fp10,...,fp2039,fp2040,fp2041,fp2042,fp2043,fp2044,fp2045,fp2046,fp2047,fp2048
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
jit_test = pd.DataFrame(test_fps)

In [28]:
jit_test.shape

(837, 2048)

In [29]:
import pandas as pd

# Assuming your DataFrame is named "jit_train"
num_columns = 2048

# Create a list of column names
column_names = [f'fp{i+1}' for i in range(num_columns)]

# Assign the column names to the DataFrame
jit_test.columns = column_names

# Verify the column names
print(jit_test.columns)


Index(['fp1', 'fp2', 'fp3', 'fp4', 'fp5', 'fp6', 'fp7', 'fp8', 'fp9', 'fp10',
       ...
       'fp2039', 'fp2040', 'fp2041', 'fp2042', 'fp2043', 'fp2044', 'fp2045',
       'fp2046', 'fp2047', 'fp2048'],
      dtype='object', length=2048)


In [36]:
import pandas as pd

# Assuming you have DataFrames named "jit_train" and "jit_activity"

# Concatenate the DataFrames vertically (along rows)
concatenated_df = pd.concat([jit_train, jit_activity], axis=1)

# Reset the index of the concatenated DataFrame
#concatenated_df.reset_index(drop=True, inplace=True)

# Verify the concatenated DataFrame
print(concatenated_df.head())

   fp1  fp2  fp3  fp4  fp5  fp6  fp7  fp8  fp9  fp10  ...  fp2040  fp2041  \
0    0    0    0    0    0    0    0    0    0     0  ...       0       0   
1    0    0    0    0    0    0    0    0    0     0  ...       0       0   
2    0    1    0    0    0    0    0    0    0     0  ...       0       0   
3    0    1    0    0    0    0    0    0    0     0  ...       0       0   
4    0    1    0    0    0    0    0    0    0     0  ...       0       0   

   fp2042  fp2043  fp2044  fp2045  fp2046  fp2047  fp2048  Activity  
0       0       0       0       0       0       0       0      2.28  
1       0       0       0       0       0       0       0     -0.99  
2       0       0       0       0       0       0       0      2.37  
3       0       0       0       0       0       0       0      1.80  
4       0       0       0       0       0       0       0      0.60  

[5 rows x 2049 columns]


In [37]:
concatenated_df.head()

,fp1,fp2,fp3,fp4,fp5,fp6,fp7,fp8,fp9,fp10,...,fp2040,fp2041,fp2042,fp2043,fp2044,fp2045,fp2046,fp2047,fp2048,Activity
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.28
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.99
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.37
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.80
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.60


In [39]:
import pandas as pd

# Assuming you have a DataFrame named "combined_df" with columns "fp1" to "fp2048" and a column "Activity"

# Extract the features (X) from combined_df
X = concatenated_df.loc[:, 'fp1':'fp2048']

# Extract the target variable (y) from combined_df
y = concatenated_df.iloc[:, -1]

# Verify the shapes of X and y
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")


Shape of X: (3351, 2048)
Shape of y: (3351,)


In [40]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [42]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
models = [
    ('Linear Regression', LinearRegression()),
    ('Decision Tree Regression', DecisionTreeRegressor()),
    ('Random Forest Regression', RandomForestRegressor())
]

In [44]:
param_grids = {
    'Linear Regression': {},
    'Decision Tree Regression': {'max_depth': [None, 10, 20]},
    'Random Forest Regression': {'n_estimators': [100, 200], 'max_depth': [None, 10, 20]}
}

In [45]:
best_model = None
best_model_name = ""
best_model_r2 = -float('inf')

for name, model in models:
    param_grid = param_grids[name]

    # Perform hyperparameter tuning using GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2')
    grid_search.fit(X_train, y_train)

    # Evaluate the model on the validation set
    y_val_pred = grid_search.predict(X_val)
    r2 = r2_score(y_val, y_val_pred)
    print(f"{name} R^2 Score: {r2}")

    # Update the best model if the current model performs better
    if r2 > best_model_r2:
        best_model_r2 = r2
        best_model = grid_search.best_estimator_
        best_model_name = name

print(f"Best Model: {best_model_name}")

Linear Regression R^2 Score: -1.241610380989785e+22
Decision Tree Regression R^2 Score: 0.27027400688487
Random Forest Regression R^2 Score: 0.48705795117987194
Best Model: Random Forest Regression


In [46]:
y_test_pred = best_model.predict(jit_test)
#test_r2 = r2_score(test_targets, y_test_pred)
#print(f"Test Set R^2 Score: {test_r2}")

In [47]:
y_test_pred

array([ 2.1219    ,  0.0208    ,  3.0931    ,  1.6721    ,  2.568     ,
        1.7408    ,  0.591075  ,  0.69915833,  2.438     ,  1.6722    ,
        2.8986    ,  3.1997    ,  2.77036667,  2.54865   ,  2.1452    ,
        1.8768    ,  1.13606667,  3.1205    ,  2.1864    ,  0.8248    ,
        1.6632    ,  1.9405    ,  1.7886    ,  2.62835   ,  0.4343    ,
        2.0114    ,  3.4792    ,  1.676     ,  0.35      ,  2.8187    ,
        2.0651    ,  2.17223333,  2.65411167,  0.82674667, -0.2156    ,
        1.8388    ,  3.1793125 ,  1.852325  ,  2.5585    ,  2.76876667,
        2.2394    ,  2.0566    ,  1.5371    ,  2.4485    ,  1.0415    ,
        3.139375  ,  3.2503    ,  2.8725    ,  1.2978    ,  3.2457    ,
        0.9532    ,  3.02198667, -0.182     ,  2.9974    ,  2.2496    ,
        1.9255    ,  1.7264    ,  2.83253333,  3.3955    ,  2.3551    ,
        2.3551    ,  2.9646    ,  2.0707    ,  2.36      ,  2.0195    ,
        2.40675833,  0.4103    ,  3.17455   ,  2.82733333,  2.37

In [48]:
len(y_test_pred)

837